<a href="https://colab.research.google.com/github/binram077/genetic-algorithm-from-scratch/blob/main/4inarowevoltution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this project is not so good at playing but I built the algorithm from scratch so you'll need to pardon me

In [ ]:
import tensorflow as tf
import numpy as np
import keras
from keras import layers

# building the 4 in a row simulation

In [ ]:
import random

DictionaryPoint = {"negative":0.0,"positive":1.0,"teko":0.5}
players = ["negative","positive",]

class board():
  def __init__(self):
    self.Boardarray = np.full((6,7),0.0)
    self.turn = random.choice([1.0,-1.0])

  def new_board(_board = None,_turn = None):
    n_board = board()
    n_board.Boardarray = _board
    n_board.turn = _turn
    return n_board


  def nextStates(self):
    States = []
    #one state for each column -> len(self.Boardarray.T)
    for i in range(len(self.Boardarray.T)):
      #check the column is not full
      if self.Boardarray[0][i] is not 0:
        s = self.Boardarray.copy()
        top = -1
        #run on the column to find the last row avialable
        for j in range(len(s)):
          if s[j][i] == 0:
            top = j
        #put the token in the row
        if top != -1:
          s[top][i] = self.turn
          States.append((board.new_board(s,self.turn * -1),i))
    return States

  #validated
  def sub_check(self,x):
    box_type = 0
    sum = 0
    for i in x:
      if box_type == i:
        if box_type != 0:
          sum += 1
      else:
        box_type = i
        sum = 0
      if sum == 3:
        return box_type
    return 0.0

  def play(self,i):
    flag = True
    for x in self.nextStates():
      if i == x[1]:
        self.Boardarray = np.copy(x[0].Boardarray)
        self.turn *= -1
        flag = False
    if flag:
      print("Error this column is full")

  def p_rand(self):
    i = random.choice(range(6))
    self.play(i)

  def CheckWin(self):
    #column and rows
    sub_check_rows_and_columns = [self.sub_check(x) for x in self.Boardarray] + [self.sub_check(x) for x in self.Boardarray.T]
    sub_check_diagonals = [self.sub_check(x) for x in [np.diag(self.Boardarray,i) for i in range(-5,7)]]+[self.sub_check(x) for x in [np.diag(np.fliplr(self.Boardarray),i) for i in range(-5,7)]]
    sum_subchecks = sub_check_rows_and_columns + sub_check_diagonals
    for x in sum_subchecks:
      if x != 0:
        return x
    return 0.0

checking the board

In [ ]:
b = board()
while b.CheckWin() == 0:
  print(b.Boardarray)
  print(b.turn)
  b.p_rand()
print(b.Boardarray)

In [ ]:
b = board()
print(b.Boardarray)
print(b.turn)
n_s = b.nextStates()
for x in n_s:
  print(x[0].Boardarray)
  print(x[0].turn)

# building the neurons, the weights and the agent


1.   the neuron is just a container of a number which symbolized how much it fires.
2.   the weights are the action when a weight is getting activated it cause the output to fire or unfire as a result to the input neuron.
3. the agent contains a neural network and he knows how to evaluate a state and playing accordingly. he also know how to mutate.

In [ ]:
from math import tanh

class container():
  def __init__(self,_number):
    self.x = 0
    self.number = _number

  def clear(self):
    self.x = 0

class entrance(container):
  def __init__(self,_number):
    container.__init__(self,_number)
  
  def update(self,_x):
    self.x = _x
  
  def output(self):
    return self.x

class neuron(container):
  def __init__(self,_number, _bias = 0):
    container.__init__(self,_number)
    self.bias = _bias

  def get(self,_x):
    self.x += _x
  
  def output(self):
    return tanh(self.x / 3) * 3 + self.bias

  def ToStr(self):
    return "neuron number " + str(self.number) + " has a bias of " + str(self.bias)

class weight():
  def __init__(self,_scalar,_input,_output):
    self.scalar = _scalar
    self.input = _input
    self.output = _output

  def activate(self,Print = False):
    self.output.get(self.input.output() * self.scalar)

  def ToStr(self):
    return "from "+str(self.input.number) + " to " + str(self.output.number) + " with scalar equal to "+str(self.scalar)



class agent():
  def __init__(self):
    self.entrances = []
    for i in range(44):
      self.entrances.append(entrance(i))
    self.middle_layer = []
    self.output = neuron(44)
    self.weights = []
    self.num_of_neurons = 45
  
  #validated, but also need to get his color from system
  def get_board(self,_board):
    for i in range(42):
      self.entrances[i].update(_board.Boardarray[i % 6][int(i/6)])
    self.entrances[42].update(_board.turn)
  
  def eval(self,_board):
    self.get_board(_board)
    for w in self.weights:
      w.activate()
    val = self.output.output()
    self.clear()
    return val
  
  def add_w(self,sc = None,_input = None, _output = None):
    if len(self.weights) >= 3000:
      return
    if sc == None:
      sc = random.uniform(-3.0,3.0)
    if _input == None:
      _input = random.choice(self.middle_layer+self.entrances)
    if _output == None:
      _output = random.choice(self.middle_layer+[self.output])
    w = weight(sc,_input,_output)
    self.weights.append(w)
  
  def add_n(self):
    if self.num_of_neurons >= 200:
      print("error too much neurons")
      return
    n = neuron(self.num_of_neurons, random.random())
    self.num_of_neurons += 1
    for i in range(3):
      self.add_w(None,None,n)
    self.add_w(None,n,None)
    self.middle_layer.append(n)
  
  def del_w(self):
    if len(self.weights) > 0:
      self.weights.remove(random.choice(self.weights))

  def change_sc(self):
    if len(self.weights) > 0:
      w = random.choice(self.weights)
      w.scalar = random.uniform(-3.0,3.0)

  def change_bias(self):
    if len(self.middle_layer) > 0:
      n = random.choice(self.middle_layer)
      n.bias = random.random()

  def clear(self):
    for x in self.entrances + self.middle_layer + [self.output]:
      x.clear()
  
  def copy(self):
    son = agent()
    for n in self.middle_layer:
      _n = neuron(son.num_of_neurons, random.random())
      _n.bias = n.bias
      son.num_of_neurons += 1
      son.middle_layer.append(_n)
    for w in self.weights:
      in_node = son.get_neuron_number_x(w.input.number)
      out_node = son.get_neuron_number_x(w.output.number)
      _scalar = w.scalar
      son.add_w(_scalar,in_node,out_node)
    return son


  def get_neuron_number_x(self,x):
    if x < 0:
      print("Error you can't ask for neuron with a negative number")
      return None
    if x < 44:
      return self.entrances[x]
    if x == 44:
      return self.output
    return self.middle_layer[x-45]
  
  def mutate(self):
    options = [agent.add_n,agent.add_w,agent.change_bias,agent.change_sc,agent.del_w]
    #the number of mutation is determined randomally with the biggest chance to one mutation at a time
    funcs = random.choices(options,[4*(200-self.num_of_neurons),3000-len(self.weights),40,70,40],k= random.randint(0,1)+random.randint(0,1) + int((random.randint(0,1)+random.randint(0,1))/2))
    for f in funcs:
     f(self)

  def mutation(self):
    son = self.copy()
    son.mutate()
    return son

  def show(self):
    print("weights:")
    for w in self.weights:
      print(w.ToStr())
    print("neurons:")
    for n in self.middle_layer+[self.output]:
      print(n.ToStr())

  def choose_move(next_boards_with_indexes):
    max_idx = 0
    max_prediction = None

  def set_color(self,x):
    self.entrances[-1].update(x) 

## the system object simulate the evolution over the generation by letting it agent to fight each other and terminating the losers

In [ ]:
class system():
  def __init__(self):
    self.agents = []
    for i in range(200):
      self.agents.append(agent())
      for j in range(3):
        self.agents[-1].add_n()
    self.champions = []
  
  def two_agent_fight(self,a,b,ToPrint = False):
    fight_board = board()
    competitors = [a,b]
    random.shuffle(competitors)
    competitors_and_colors = [[competitors[0],fight_board.turn],[competitors[1],fight_board.turn * -1]]
    current_player_idx = 0
    while fight_board.CheckWin() == 0:
      if len(fight_board.nextStates()) == 0:
        self.champions.append(random.choice(competitors))
        return
      chosen_state_number = None
      chosen_state_evaluation = -5
      for n_s in fight_board.nextStates():
        competitors_and_colors[current_player_idx][0].set_color(competitors_and_colors[current_player_idx][1])
        evaluation = competitors_and_colors[current_player_idx][0].eval(n_s[0])
        if evaluation > chosen_state_evaluation:
          chosen_state_evaluation = evaluation
          chosen_state_number = n_s[1]
      fight_board.play(chosen_state_number)
      if ToPrint:
        print(fight_board.Boardarray)
        print("")
      current_player_idx = 1 - current_player_idx
    if fight_board.CheckWin() == 0.5:
      self.champions.append(random.choice([a,b]))
      return
    self.champions = self.champions + [x[0] for x in competitors_and_colors if x[1] == fight_board.CheckWin()]

  def next_gen(self,PrintFirst = False):
    random.shuffle(self.agents)
    while len(self.agents) >= 2:
      a = self.agents.pop()
      b = self.agents.pop()
      self.two_agent_fight(a,b,PrintFirst)
      PrintFirst = False
    self.agents = [x.mutation() for x in self.champions] + self.champions
    self.champions = []

  def next_gens(self,n):
    for i in range(n):
      self.next_gen()
      if i % 5 == 0:
        print("this is the " + str(i) + "th  generation")
        print("------------------------------\n------------------------------")
        self.next_gen(True)

testing the board with the agent

In [ ]:
B = board()
for i in range(6):
  B.p_rand()
print(B.Boardarray)
A = agent()
A.add_n()
A.add_n()
A.show()
Ason = A.mutation()
Ason.show()

print("evaluation:")
print(A.eval(B))
print("evaluation 2:")
print(Ason.eval(B))



In [ ]:
A = agent()
A.add_n()
A.add_n()
A.add_n()
B = agent()
B.add_n()
B.add_n()
B.add_n()
S = system()
S.two_agent_fight(A,B,True)
print(S.champions[0] is A)

## the game object is the user interface that let the human player play against one of the agents

In [ ]:
class game():
  def __init__(self,A):
    self.Game_agent = A
    self.Game_board = board()
    print("you play the 1 color, would you prefer to start? answer y for yes and n for no")
    letter = input()
    if letter == 'y':
      self.Game_board.turn = 1.0
      return
    if letter == 'n':
      self.Game_board.turn = -1.0
      return
    else:
      print("you can't choose this option")

  def start(self):
    while self.Game_board.CheckWin() == 0:
      print(self.Game_board.Boardarray)
      if len(self.Game_board.nextStates()) == 0:
        print("it's a draw!")
        return
      if self.Game_board.turn == 1.0:
        print("enter your move(a number of a column)")
        self.Game_board.play(int(input())-1)
      elif self.Game_board.turn == -1.0:
        idx = -1
        max_eval = -5
        for x in self.Game_board.nextStates():
          self.Game_agent.set_color(-1.0)
          cur_eval = self.Game_agent.eval(x[0])
          if cur_eval > max_eval:
            max_eval = cur_eval
            idx = x[1]
        self.Game_board.play(idx)
    print(self.Game_board.Boardarray)
    if self.Game_board.CheckWin() == 1.0:
      print("you won")
    if self.Game_board.CheckWin() == -1.0:
      print("agent won")

checking the game and the system and playing against the computer

In [ ]:
s = system()
s.next_gens(1200)

In [ ]:
g = game(s.agents[0])
g.start()